In [1]:
pip install tweepy

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/98.5 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/98.5 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/98.5 kB ? eta -:--:--
   ------------ --------------------------- 30.7/98.5 kB 262.6 kB/s eta 0:00:01
   --------------------------------- ------ 81.9/98.5 kB 508.4 kB/s eta 0:00:01
   ---------------------------------------- 98.5/98.5 kB 565.9 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install textblob

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import tweepy
import re
from textblob import TextBlob

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

# Sample dataset of tweets with sentiment labels (0 for negative, 1 for positive)
tweets = [
    ("I hate candidate A", 0),
    ("I love candidate B", 1),
    ("Candidate A is terrible", 0),
    ("I support candidate B", 1),
    ("Can't stand candidate A", 0),
    ("I'm voting for candidate B", 1)
]

# Convert the dataset into a DataFrame
df = pd.DataFrame(tweets, columns=['text', 'sentiment'])

# Preprocessing function
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenize the text
    words = word_tokenize(text)
    # Remove stopwords
    words = [word for word in words if word not in stopwords.words('english')]
    # Return the preprocessed text as a single string
    return ' '.join(words)

# Apply preprocessing to the 'text' column
df['text'] = df['text'].apply(preprocess_text)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['sentiment'], test_size=0.2, random_state=42)

# TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=1000)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Train a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train_vectorized, y_train)

# Make predictions
y_pred = rf_classifier.predict(X_test_vectorized)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Print classification report
print(classification_report(y_test, y_pred))

# Create a DataFrame to store the sentiment scores for each tweet
sentiment_df = pd.DataFrame({'text': X_test, 'sentiment': y_test, 'predicted_sentiment': y_pred})

# Add a column to indicate the candidate mentioned in each tweet
sentiment_df['candidate'] = sentiment_df['text'].apply(lambda x: 'A' if 'candidate a' in x else ('B' if 'candidate b' in x else None))

# Calculate the average sentiment score for each candidate
average_sentiment = sentiment_df.groupby('candidate')['predicted_sentiment'].mean()

# Determine the candidate with the highest average sentiment score
winning_candidate = average_sentiment.idxmax()

print(f"Candidate {winning_candidate} has the highest possibility of winning.")

Accuracy: 0.5
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.50      1.00      0.67         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2

Candidate B has the highest possibility of winning.


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
